In [1]:
import numpy as np
import pandas as pd
import re
from datetime import timedelta 
import sys

def get_seconds(time_str):
    hh, mm, ss = time_str.split(':')
    return int(hh) * 3600 + int(mm) * 60 + int(ss)

# Below code is used for extracting required fields and cleaning the data 


In [2]:
pattern_serial_number = "^Serial Number:"
pattern_epoch_time ="^Epoch Period"
pattern_date = "^Date"
date_found = False #State Variable 
serial_number = ""
epoch_time = ""
customer_data_file = open("customer_data.csv","w+")

participant_id = input("Please Type the Participant ID as per the saved File:")
folder_name = participant_id[:-14]
#file_name = '60sDataTable'
    
print(folder_name)

# Open the file with read only permit
f = open('C:\\Users\\slinga4\\Desktop\\E3 Folder\\'+folder_name+'\\'+participant_id+'.csv')         
# use readline() to read the first line 
line = f.readline()
# use the read line to read further.
# If the file is not empty keep reading one line
# at a time, till the file is empty
while line:
    # in python 2+
    # print line
    # in python 3 print is a builtin function, so
    
    result = re.match(pattern_serial_number, line)
    if result:
        serial_number = line
        
    result = re.match(pattern_epoch_time, line)
    if result:
      epoch_time = line
    
    result = re.match(pattern_date, line)
    if result:
      date_found = True 
    
    if date_found == True:
        customer_data_file.write(line)
        
    # use readline() to read next lineS
    line = f.readline()
    
f.close()
customer_data_file.close()
print(serial_number.replace(",",""))
serial_number = serial_number.replace(",","")
print(epoch_time.replace(",",""))
epoch_time = epoch_time.replace(",","")
epoch_time_calculation = get_seconds(epoch_time.split(' ')[3])
print(epoch_time_calculation)

Please Type the Participant ID as per the saved File:Samarth (2021-10-08)60secDataTable
Samarth (2021-10-08)
Serial Number: MOS2E47200163

Epoch Period (hh:mm:ss) 00:01:00

60


# Below code is used for validating the time frame from the device


In [3]:
new_data = pd.read_csv("customer_data.csv")
days = new_data["Date"].unique()
for day in days: 
    filtered_days = new_data[new_data["Date"] == day]
    diff = 0
    diff_array = []
    first_row_read = False
    for index,row in filtered_days.iterrows(): 
        if diff==0 and first_row_read == False:
            first = get_seconds(row[" Time"])
            first_row_read = True
        else:
            diff = get_seconds(row[" Time"]) - first 
            first = get_seconds(row[" Time"])
            diff_array.append(diff)
    print(np.average(diff_array))
    if epoch_time_calculation == int(np.average(diff_array)):
        print("Sample Matched")
    else:
        print("Sample Did not Match")

60.0
Sample Matched
60.0
Sample Matched
60.0
Sample Matched
60.0
Sample Matched
60.0
Sample Matched
60.0
Sample Matched
60.0
Sample Matched
60.0
Sample Matched
60.0
Sample Matched
60.0
Sample Matched
60.0
Sample Matched
60.0
Sample Matched
60.0
Sample Matched
60.0
Sample Matched
60.0
Sample Matched
60.0
Sample Matched
60.0
Sample Matched
60.0
Sample Matched


# Below code contains the Logic for calculation of compensation amount

In [4]:
new_data = pd.read_csv("customer_data.csv")
stats = {}
days = new_data["Date"].unique()
compensation_amount_list = []
amt = 0 
new_dict = {}

for day in days: 
    stats[day] = 0
    filtered_days = new_data[new_data["Date"] == day]
    for index,row in filtered_days.iterrows(): 
        if row["Inclinometer Standing"]!=0 or row["Inclinometer Sitting"]!=0 or row["Inclinometer Lying"]!=0:
            stats[day] += 1
    
for day in days:
    print("Date:", day)
    #print("no_of_rows: ",stats[day])
    total_seconds = stats[day]*epoch_time_calculation
    #print(total_seconds)
    print("Total time worn for that day:",str(timedelta(seconds = total_seconds)))
    if total_seconds < (2*60*60):
            compensation_amount = 0
            print("compensation amount for the day:", compensation_amount)
    else:
        compensation_amount = 15
        print("compensation amount for the day:", compensation_amount)
    compensation_amount_list.append(compensation_amount)
    new_dict[day] = [str(timedelta(seconds = total_seconds)), compensation_amount]
    
for amount in compensation_amount_list:
    amt += amount
print("Total Amount to be paid: $",amt)
#print(new_dict)

Date: 9/21/2021
Total time worn for that day: 3:34:00
compensation amount for the day: 15
Date: 9/22/2021
Total time worn for that day: 2:33:00
compensation amount for the day: 15
Date: 9/23/2021
Total time worn for that day: 2:09:00
compensation amount for the day: 15
Date: 9/24/2021
Total time worn for that day: 3:50:00
compensation amount for the day: 15
Date: 9/25/2021
Total time worn for that day: 3:15:00
compensation amount for the day: 15
Date: 9/26/2021
Total time worn for that day: 3:04:00
compensation amount for the day: 15
Date: 9/27/2021
Total time worn for that day: 3:13:00
compensation amount for the day: 15
Date: 9/28/2021
Total time worn for that day: 1:16:00
compensation amount for the day: 0
Date: 9/29/2021
Total time worn for that day: 2:33:00
compensation amount for the day: 15
Date: 9/30/2021
Total time worn for that day: 0:00:00
compensation amount for the day: 0
Date: 10/1/2021
Total time worn for that day: 0:05:00
compensation amount for the day: 0
Date: 10/2/20

# Below Code Gives the output in a Text File

In [5]:
import sys
import os.path


original_stdout = sys.stdout 
save_path = 'C:\\Users\\slinga4\\Desktop\\E3 Folder\\'+folder_name 
filename_amt = os.path.join(save_path, folder_name+"_total_incentive"+".csv")         

#filename_amt = participant_id + '.txt'

with open(filename_amt, 'w') as f:
    sys.stdout = f 
    print("Serial Number of Accelerometer:", serial_number)
    print("Participant ID:", participant_id)
    for key,value in new_dict.items():
        print("Date: ", key)
        print("Time Worn in a Day:", value[0])
        print("Amount for the day: ", value[1])
        print('\n')
    print("Total Amount to be paid: $",amt)
    
    sys.stdout = original_stdout 